### Import packages

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.cloud import bigquery_storage
from sklearn.linear_model import LinearRegression
from datetime import datetime
import warnings
warnings.filterwarnings(action="ignore")

### User-defined global inputs

In [88]:
billing_project = "logistics-customer-staging" # The project that is displayed at the top of your BQ console
dataset = "cl" # For Global teams, use "cl". For local and regional teams, use "curated_data_shared"

# Entity of choice
entities_of_choice = ["HS_SA"] # Choose the entity ID(s) that have the tests you want to analyze

# You can define a custom test list here. If you don't, the script will analyze ALL AB tests in the country of choice with the word elasticity in them.
is_use_defined_test_names = True # Set to True if you want to use the custom test list below
custom_test_list = ["SA_20230712_R_00_O_Riyadh_ Locals elasticity test"] # Define the custom tests you want to analyze here

# Test keyword
test_keyword_list = ["elasticity", "customer_location"] # If "is_use_defined_test_names" = False, the script will analyze all tests with these keywords in the test name

# Grouping method
# We will use two grouping methods, "pct_ranks" and "avg"
# For the "pct_ranks" method, you need to specify the number of percentile ranks you want to use
# For the "avg" method, you need to define the *min* and *max* thresholds
    # Areas with a slope below the *min* threshold x avg slope of all areas would be labeled as "low elasticity" areas
    # Areas with a slope above the *max* threshold x avg slope of all areas would be labeled as "high elasticity" areas
    # Areas with slopes between *min* threshold x avg slope and *max* threshold x avg slope would be labeled as "mid elasticity" areas
num_groups = 3 # Only applicable for the "pct_ranks" grouping_method
slope_lower_threshold = 0.8 # Only applicable for the "avg" grouping method. Areas with a slope of (1 - slope_lower_threshold)% below average would be labeled as "low elasticity" areas
slope_upper_threshold = 1.2 # Only applicable for the "avg" grouping method. Areas with a slope (slope_upper_threshold - 1)% above average would be labeled as "high elasticity" areas

#############################################################################################################################################  
**<center>#--------STARTING HERE, YOU DON'T NEED TO DEFINE ANY OTHER INPUTS--------#</center>**  
#############################################################################################################################################

### Instantiate a BQ client

In [3]:
client = bigquery.Client(project=billing_project)
bqstorage_client = bigquery_storage.BigQueryReadClient()

### Pull the names of the elasticity tests

In [4]:
test_name_query = """
    SELECT DISTINCT
        entity_id,
        test_name,
        test_id,
        test_start_date,
        test_end_date,
        experiment_type
    FROM `fulfillment-dwh-production.{dataset}.dps_experiment_setups`
    WHERE LOWER(experiment_type) = "ab" AND entity_id IN ("{entities_of_choice}")
    ORDER BY test_end_date DESC
""".format(
    dataset=dataset,
    entities_of_choice='", "'.join(entities_of_choice)
)

df_test_names = pd.DataFrame(client.query(query=test_name_query).result().to_dataframe(progress_bar_type="tqdm", bqstorage_client=bqstorage_client))

# If is_use_defined_test_names == True, overwrite list_test_names with the custom_test_list. If it is False, filter the test names by the test_keyword_list
if is_use_defined_test_names == True:
    df_test_names = df_test_names[df_test_names["test_name"].isin(custom_test_list)]
else:
    df_test_names = df_test_names[df_test_names["test_name"].str.contains("|".join(test_keyword_list), case=False)]

# Create a list of test names out of df_test_names
list_test_names = df_test_names["test_name"].tolist()
list_test_ids = df_test_names["test_id"].tolist()

# Define the lookback window for the analysis based on the minimum test start date
lookback_period_start_date = datetime.strftime(df_test_names["test_start_date"].min().date(), "%Y-%m-%d")
lookback_period_end_date = datetime.strftime(df_test_names["test_end_date"].max().date(), "%Y-%m-%d")

Downloading: 100%|██████████|


### Pull the test data

In [5]:
test_data_query = """
	-- Get the GADM data for the countries of choice
	WITH gadm_data AS (
		SELECT 
			ent.segment AS region,
			ent.global_entity_id AS entity_id,
			gadm.*
		FROM `logistics-data-storage-staging.long_term_pricing.gadm_geo_spatial_data_level_2` gadm
		LEFT JOIN `fulfillment-dwh-production.curated_data_shared_coredata.global_entities` ent ON gadm.gid_0 = ent.country_iso_a3
		WHERE TRUE
			AND ent.is_reporting_enabled
            AND ent.is_entity_online
            AND ent.global_entity_id IN ("{entities_of_choice}")
            AND ent.brand_name IN (
				"Foodpanda",
				"Baemin",
				"Talabat",
				"Yemeksepeti",
				"HungerStation",
				"eFood",
				"Foodora",
				"PedidosYa"
			)
	),
    
    -- Get the perseus location of each perseus session ID for the tests of choice
    perseus_location AS (
		SELECT
			x.entity_id,
			x.perseus_session_id,
			x.perseus_location
		FROM `fulfillment-dwh-production.{dataset}.dps_sessions_mapped_to_perseus_sessions` x
		LEFT JOIN UNNEST(events) AS e
		LEFT JOIN `fulfillment-dwh-production.{dataset}.dps_experiment_setups` tst ON x.entity_id = tst.entity_id AND x.sessions.experiment_id = tst.test_id
		WHERE TRUE
			AND x.created_date BETWEEN DATE("{lookback_period_start_date}") AND DATE("{lookback_period_end_date}") -- Filter for the right lookback window
			AND tst.test_name IN ("{test_names}") -- Filter for the right tests
			AND x.sessions.variant != "Original" -- Exclude orders from ASAs
			AND e.vertical_parent_in_test = TRUE -- Only include sessions that have the same parent vertical as the one configured in the test
		QUALIFY ROW_NUMBER() OVER (PARTITION BY x.entity_id, x.perseus_session_id ORDER BY e.event_time DESC) = 1 -- Get the latest location for every session
	),
    
    -- Enrich the perseus location data with the GADM data
    perseus_location_enriched AS (
		SELECT
			x.*,
			gadm.geometry AS admin_area_geometry,
			gadm.name_2 AS admin_area_name
		FROM perseus_location x
        -- This LEFT JOIN checks if the customer location is contained within an admin area and returns TRUE if that is the case
		LEFT JOIN gadm_data gadm ON x.entity_id = gadm.entity_id AND ST_CONTAINS(SAFE.ST_GEOGFROM(gadm.geometry), x.perseus_location)
    ),

    -- Calculate CVR and CVR3 for each test-area-variant combination
    cvr_agg_data AS (
		SELECT
			ent.segment AS region,
			x.entity_id,
			x.test_name,
			loc.admin_area_geometry,
			loc.admin_area_name,
			x.variant AS test_variant,
			COUNT(DISTINCT x.fe_session_id) AS total_session_count,
			COALESCE(ROUND(COUNT(DISTINCT x.transaction_no) / NULLIF(COUNT(DISTINCT x.fe_session_id), 0), 5), 0) AS cvr,
			COALESCE(ROUND(COUNT(DISTINCT x.transaction_no) / NULLIF(COUNT(DISTINCT x.shop_menu_no), 0), 5), 0) AS cvr3
		FROM `fulfillment-dwh-production.{dataset}.dps_test_cvr_treatment` x
		LEFT JOIN perseus_location_enriched loc ON x.entity_id = loc.entity_id AND x.fe_session_id = loc.perseus_session_id
		INNER JOIN `fulfillment-dwh-production.curated_data_shared_coredata.global_entities` ent ON x.entity_id = ent.global_entity_id -- Get the region associated with every entity_id
		WHERE TRUE
			AND x.created_date BETWEEN DATE("{lookback_period_start_date}") AND DATE("{lookback_period_end_date}") -- Filter for the right lookback window
			AND x.test_name IN ("{test_names}") -- Filter for the right tests
			AND x.variant != "Original" -- Exclude orders from ASAs
			AND x.vertical_parent_in_test = TRUE -- Only include sessions that have the same parent vertical as the one configured in the test
		GROUP BY 1, 2, 3, 4, 5, 6
    ),
    
    -- Calculate the avg DF for each test-area-variant combination
    orders_agg_data AS (
		SELECT
        	ent.segment AS region,
			a.entity_id,
			b.test_name,
            gadm.geometry AS admin_area_geometry,
  			gadm.name_2 AS admin_area_name,
			a.test_variant,
            AVG(a.delivery_fee_local) AS avg_df_local,
		FROM `fulfillment-dwh-production.{dataset}.dps_sessions_mapped_to_orders` a
		LEFT JOIN `fulfillment-dwh-production.{dataset}.dps_test_orders` b ON a.entity_id = b.entity_id AND a.order_id = b.order_id
        -- This LEFT JOIN checks if the customer location is contained within an admin area and returns TRUE if that is the case
		LEFT JOIN gadm_data gadm ON a.entity_id = gadm.entity_id AND ST_CONTAINS(SAFE.ST_GEOGFROM(gadm.geometry), a.customer_location)
        INNER JOIN `fulfillment-dwh-production.curated_data_shared_coredata.global_entities` ent ON a.entity_id = ent.global_entity_id -- Get the region associated with every entity_id
		WHERE TRUE
			AND a.created_date BETWEEN DATE("{lookback_period_start_date}") AND DATE("{lookback_period_end_date}")
			AND b.test_name IN ("{test_names}")
			AND a.is_own_delivery -- OD orders only
			AND a.test_variant != "Original" -- Exclude orders from ASAs
			AND a.is_match_test_vertical
		GROUP BY 1, 2, 3, 4, 5, 6
    )
    
    -- Join the CVR and order tables
    SELECT
		cvr.region,
		cvr.entity_id,
		cvr.test_name,
        cvr.admin_area_geometry,
        cvr.admin_area_name,
		cvr.test_variant,
		cvr.total_session_count,
		cvr.cvr,
		cvr.cvr3,
		ord.avg_df_local
	FROM cvr_agg_data cvr
	-- We use an INNER JOIN because we only want to include areas that have both CVR and order data
    INNER JOIN orders_agg_data ord USING(entity_id, test_name, admin_area_geometry, admin_area_name, test_variant)
	ORDER BY 1, 2, 3, 4
""".format(
    dataset=dataset,
    lookback_period_start_date=lookback_period_start_date,
    lookback_period_end_date=lookback_period_end_date,
    entities_of_choice='", "'.join(entities_of_choice),
    test_names = '", "'.join(list_test_names)
)

df_test_data = pd.DataFrame(client.query(query=test_data_query).result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm"))\
    .sort_values(by=["region", "entity_id", "test_name", "admin_area_name", "test_variant"])\
    .reset_index(drop=True)

# Print the length of the data frame
print(f"The length of the data frame is: {len(df_test_data)}")

# Display the head of df_test_data
df_test_data.head(10)

Downloading: 100%|██████████|
The length of the data frame is: 16


,region,entity_id,test_name,admin_area_geometry,admin_area_name,test_variant,total_session_count,cvr,cvr3,avg_df_local
0,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Control,904,0.42035,0.44496,17.483854893
1,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation1,885,0.40904,0.43614,19.211262413
2,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation2,825,0.37939,0.40649,18.407670676
3,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation3,923,0.39003,0.41715,16.593374098
4,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation4,893,0.39082,0.41847,15.653652084
5,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,"POLYGON ((46.07978999900007 24.9505999000001, ...",Duruma,Control,1,0.00000,0.00000,24.100000000
6,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,"POLYGON ((46.07978999900007 24.9505999000001, ...",Duruma,Variation3,1,0.00000,0.00000,26.214285714
7,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,"POLYGON ((46.07978999900007 24.9505999000001, ...",Duruma,Variation4,3,1.00000,1.00000,26.562500000
8,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.46406000000019 25.016969900000106...,Huraymila,Control,1,0.00000,0.00000,35.000000000
9,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.46406000000019 25.016969900000106...,Huraymila,Variation1,1,1.00000,1.00000,37.000000000


### Calculate the slope of the linear regression line of CVR Vs. Avg DF per test-area combination

In [6]:
# Change the data types of the columns
df_test_data.loc[:, "total_session_count":] = df_test_data.loc[:, "total_session_count":].apply(lambda x: pd.to_numeric(x))

# Calculate the *absolute change in CVR* and the *percentage change in avg DF* between each variant and the control group
df_test_data_enriched = df_test_data.copy()

# Calculate the absolute change in CVR
df_test_data_enriched[["abs_cvr_change", "abs_cvr3_change"]] = df_test_data_enriched.\
    groupby(["region", "entity_id", "test_name", "admin_area_name"])[["cvr", "cvr3"]].\
    transform(lambda x: x - x.iloc[0])

# Calculate the percentage change in avg DF
df_test_data_enriched["pct_avg_df_change"] = df_test_data_enriched.\
    groupby(["region", "entity_id", "test_name", "admin_area_name"])["avg_df_local"].\
    transform(lambda x: (x - x.iloc[0]) / x.iloc[0])

# Display the head of the data frame
df_test_data_enriched.head()

,region,entity_id,test_name,admin_area_geometry,admin_area_name,test_variant,total_session_count,cvr,cvr3,avg_df_local,abs_cvr_change,abs_cvr3_change,pct_avg_df_change
0,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Control,904,0.42035,0.44496,17.483855,0.00000,0.00000,0.0
1,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation1,885,0.40904,0.43614,19.211262,-0.01131,-0.00882,0.0988
2,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation2,825,0.37939,0.40649,18.407671,-0.04096,-0.03847,0.052838
3,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation3,923,0.39003,0.41715,16.593374,-0.03032,-0.02781,-0.050932
4,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation4,893,0.39082,0.41847,15.653652,-0.02953,-0.02649,-0.10468


In [7]:
# Create a function that regresses a variable "X" on another variable "y"
def reg_model(df, cvr_col, df_col):
    data_x = df[[df_col]].values
    data_y = df[[cvr_col]].values
    lm = LinearRegression()
    lm.fit(X=data_x, y=data_y)
    return float(np.squeeze(lm.coef_))

In [8]:
# Calculate the slope of the linear regression between the absolute change in CVR and the percentage change in avg DF per test-area combination
df_slopes = pd.DataFrame(
    df_test_data_enriched.\
        groupby(["region", "entity_id", "test_name", "admin_area_name"]).\
        apply(reg_model, cvr_col="abs_cvr_change", df_col="pct_avg_df_change"),
    columns=["reg_slope_per_test_area"]
    )

# Reset the index to flatten the data frame
df_slopes.reset_index(inplace=True)

# Join df_slopes to df_test_data_enriched
df_test_data_enriched = df_test_data_enriched.merge(
    df_slopes,
    how="left",
    on=["region", "entity_id", "test_name", "admin_area_name"]
)

# Display the head of the data frame
df_test_data_enriched.head()

,region,entity_id,test_name,admin_area_geometry,admin_area_name,test_variant,total_session_count,cvr,cvr3,avg_df_local,abs_cvr_change,abs_cvr3_change,pct_avg_df_change,reg_slope_per_test_area
0,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Control,904,0.42035,0.44496,17.483855,0.00000,0.00000,0.0,0.048451
1,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation1,885,0.40904,0.43614,19.211262,-0.01131,-0.00882,0.0988,0.048451
2,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation2,825,0.37939,0.40649,18.407671,-0.04096,-0.03847,0.052838,0.048451
3,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation3,923,0.39003,0.41715,16.593374,-0.03032,-0.02781,-0.050932,0.048451
4,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,POLYGON ((46.45037000000008 24.703139899000064...,Ad Diriyah,Variation4,893,0.39082,0.41847,15.653652,-0.02953,-0.02649,-0.10468,0.048451


### Create a function to pre-process the slope values

In [90]:
def dataset_pre_processing(test_name, column_name):
    # Filter the data frame for the region, entity_id, and test_name of choice
    df = df_test_data_enriched[df_test_data_enriched["test_name"]==test_name]

    # Filter out rows with positive slopes (even though this would be very difficult to attain in reality, assuming a proper customer area discovery test design)
    df = df[df[column_name] < 0]

    # Apply the modulus function to remaining values under the slope column to get the absolute values
    df[column_name] = df[column_name].apply(lambda x: abs(x))

    return df    

### Create a function to compute the groups if grouping_method = "pct_ranks"

In [91]:
def pct_ranks_method_func(test_name, column_name, num_groups):
    # Apply the dataset pre-processing function
    df = dataset_pre_processing(test_name=test_name, column_name=column_name)

    # Calculate the percentile ranks based the number of groups defined by the user
    percentiles = np.arange(0, 100, 100/num_groups)

    # Calculate the percentile values
    percentile_values = np.percentile(df[column_name], percentiles)
    
    # Create a data frame with the percentile ranks and values
    df_percentiles = pd.DataFrame({
        'region': df['region'].unique()[0],
        'entity_id': df['entity_id'].unique()[0],
        'test_name': df['test_name'].unique()[0],
        'slope_ntile_lower_threshold': percentiles,
        'slope_ntile_upper_threshold': pd.Series(percentiles).shift(-1).fillna(100),
        'slope_lower_threshold': percentile_values,
        'slope_upper_threshold': pd.Series(percentile_values).shift(-1).fillna(df[column_name].max())
    })

    # Create a new column that labels each row
    df_percentiles["group_name"] = df_percentiles.apply(
        lambda x: f"{str(int(x['slope_ntile_lower_threshold']))}_{str(int(x['slope_ntile_upper_threshold']))}_ntile_group", axis=1
    )
    return df_percentiles

### Create a function to compute the groups if grouping_method = "avg_method"

In [94]:
def avg_method_func(test_name, column_name, min_thr, max_thr):
    # Apply the dataset pre-processing function
    df = dataset_pre_processing(test_name=test_name, column_name=column_name)

    # Calculate the avg slope using values from the pre-processed data frame
    avg_slope = df[column_name].mean()

    # Calculate the min and max thresholds
    slope_lower_threshold = min_thr * avg_slope
    slope_upper_threshold = max_thr * avg_slope

    # Create a data frame with the percentile ranks and values
    df_avg_method = pd.DataFrame({
        'region': df['region'].unique()[0],
        'entity_id': df['entity_id'].unique()[0],
        'test_name': df['test_name'].unique()[0],
        'slope_lower_threshold': [slope_lower_threshold],
        'slope_upper_threshold': [slope_upper_threshold]
    })

    return df_avg_method

In [95]:
# Compute the groups of each test using both methods (avg_method and pct_ranks)
df_pct_ranks_method = []
df_avg_method = []
for tst in list_test_names:
    # Apply the percentile ranks method
    df_pct_ranks_method_iter = pct_ranks_method_func(test_name=tst, column_name='reg_slope_per_test_area', num_groups=num_groups)
    df_pct_ranks_method.append(df_pct_ranks_method_iter)

    # Apply the avg method
    df_avg_method_iter = avg_method_func(test_name=tst, column_name='reg_slope_per_test_area', min_thr=slope_lower_threshold, max_thr=slope_upper_threshold)
    df_avg_method.append(df_avg_method_iter)

# Concatenate the list of data frames into one data frame
df_pct_ranks_method = pd.concat(df_pct_ranks_method)
df_avg_method = pd.concat(df_avg_method)

# Display the resulting data frame
print("The resulting data frame of the pct_ranks method is:")
display(df_pct_ranks_method)

print("The resulting data frame of the avg method is:")
display(df_avg_method)

The resulting data frame of the pct_ranks method is:


,region,entity_id,test_name,slope_ntile_lower_threshold,slope_ntile_upper_threshold,slope_lower_threshold,slope_upper_threshold,group_name
0,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,0.000000,33.333333,0.082813,0.082813,0_33_ntile_group
1,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,33.333333,66.666667,0.082813,0.082813,33_66_ntile_group
2,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,66.666667,100.000000,0.082813,0.082813,66_100_ntile_group


The resulting data frame of the avg method is:


,region,entity_id,test_name,slope_lower_threshold,slope_upper_threshold
0,MENA,HS_SA,SA_20230712_R_00_O_Riyadh_ Locals elasticity test,0.066251,0.099376
